# Session 5 

Thios file inculdes the sessiopn note on session 5, where we are looking at tablua data in Pandas, logical slices and potentially making plots.
Today we are going to focus on how we can slice data in pasrticulasr ways to get the data we want, oftn befre we try to visualise.

In [ ]:
# Set up 
import pandas as pd 

# read in the excel, reading in every sheet
filepath = "https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"
data = pd.read_excel(filepath, sheet_name=None)


In [ ]:
# start to investigate the data
# you'll need to import a module we use every week

data.keys() # the dictionry keys are each of the sheet manes and the values are the dataframes 

# 
data['header']

# print into on one dataframe 
data['header'].info()

## Slicing and using rules
usding 903 rules data valadation rules for LA 

In [23]:
# .notna() and .isna 
# returns a slice in rows withourt an NA 

# 102 - date of birth is not a valud day 

header = data['header']

#header.info()
header.head()  # DOb is in format d/m/y

header['DOB'] = pd.to_datetime(header['DOB'], format="%d-%m-%Y", errors='coerce') 
# as not all dates can be changed into the datetime we ned to coerce errors 

invaid_dob = header['DOB'].isna()
error_rows = header[invaid_dob]

# or 
#invaid_dob = header['DOB'].notna()
#error_rows = header[~invaid_dob]



In [31]:
# 115 - Date of Local Authority's (LA) decision that a child should be placed for adoption is not a valid date."
# table is placed_for_adoption
# column is DATE_PLACED

adopt = data['placed_for_adoption']

#adopt.info()
adopt['DATE_PLACED'].head() 

adopt['DATE_PLACED'] = pd.to_datetime(adopt['DATE_PLACED'], format="%d/%m/%Y", errors='coerce')
invalid_date = adopt['DATE_PLACED'].isna()
error_rows = adopt[invalid_date]

error_rows

,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED
29,bad_entry_1,Not a date,NaT,This also isn't a date,oop


In [ ]:
 # .isin()
# 103 - The ethnicity code is either not valid or has not been entered.

ethncity_codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
    ]

valid_ethnicity = header['ETHNIC'].isin(ethncity_codes)

error_rows = header[~valid_ethnicity]

error_rows


In [45]:
# 114 - Data entry to record the status of former carer(s) of an adopted child is invalid.
#code_list = ["0", "1"]
# table is ad1
# the column is FOSTER_CARE

ad1 = data['ad1'] 

# ad1['FOSTER_CARE'].head()

code_list = [0,1]

valid_codes = ad1['FOSTER_CARE'].isin(code_list)

error_rows = ad1[~valid_codes]
error_rows

# when doing is in teh data types of the condition must be the sam,e. for instanecthis works because teh values and codes are both int. we could do it with strings but both must be strings.

,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
29,bad_entry,datedate,notdate,datenot,777,Q,Will,let's go


In [51]:

df = pd.merge(adopt, header, how='left', on='CHILD', suffixes=[None, "_header"])

df['DOB'] = pd.to_datetime(df['DOB'], format="%d/%m/%Y", errors='coerce')
df['DATE_PLACED'] = pd.to_datetime(df['DATE_PLACED'], format="%Y-%m-%d", errors="coerce")

place_before_birth = (df['DATE_PLACED'] >= df['DOB']) | (df['DATE_PLACED'].isna())

error_rows = df[~place_before_birth]
error_rows

,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,SEX,DOB_header,ETHNIC,UPN,MOTHER,MC_DOB


In [59]:
# 302 - First episode starts before child was born.
# tables are header and episodes
# DOB from header and DECOM from episodes

episodes = data['episodes']
episodes.head()

df = pd.merge(header, episodes, how='left', on='CHILD', suffixes=[None, "_ep"])
df['DECOM'] = pd.to_datetime(df['DECOM'], format="%d/%m/%Y", errors='coerce')

condition = df['DECOM'] < df['DOB'] 

error_rows = df[condition]
error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN


In [64]:
## sorting by the first episde 

episodes = episodes.sort_values(['CHILD','DECOM'], ascending=True)
episodes.drop_duplicates('CHILD', keep='first', inplace=True)

# here w3e sord by child id and then by Decom. From this we drop all duplicates and we are thus left with the first episode for each child 
# the inplace function lets us assign the outpu5t 5to the variable uiself withought actully assigning it 

episodes

,CHILD,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN
4801,32,12/02/2017,T,J1,N3,K2,PR1,15/04/2017,E3,NaN,ED8 1OX,WI5 9CC,7876040
4042,982,07/10/2017,T,E1,N7,H5,PR4,15/10/2017,X1,APPRR,EF24 4OP,WB15 6HK,3183884
437,1243,02/01/2017,L,V4,N5,Z1,PR0,18/05/2017,E9,NaN,EC5 5MZ,WC30 9OS,7270736
1796,1407,07/06/2017,P,L2,N3,U3,PR5,07/10/2017,X1,CUSTOD,EK9 2HR,WI24 8QE,8380497
3851,2431,01/02/2018,P,L2,N3,A3,PR5,NaN,E14,NaN,EZ17 3GY,WT6 3JM,8470650
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,997201,07/06/2017,T,L3,N4,A5,PR1,NaN,X1,NaN,EC16 9ZF,WN12 2KE,2276120
2766,997445,20/06/2016,P,J1,N2,U3,PR3,24/09/2017,E48,NaN,EP2 2VO,WE6 1FZ,3786993
155,998192,08/07/2017,T,C2,N2,U1,PR4,NaN,X1,CREQB,EL26 9FW,WA22 6DB,1505258
156,998777,11/10/2015,P,C2,N2,U1,PR2,17/03/2018,X1,CLOSE,ER24 5VS,WJ7 5PQ,1112628


The standard name for tehse conditions that were mking is error mask 

In [68]:
# 188 - Child is aged under 4 years at the end of the year but a Strengths and Difficulties (SDQ) score or a reason for no SDQ score has been completed
oc2 = data['oc2']
 
oc2['DOB'] = pd.to_datetime(oc2['DOB'], format="%d/%m/%Y", errors='coerce')
 
collection_end = pd.to_datetime('31/03/2020', dayfirst=True)
 
oc2['4bday'] = oc2['DOB'] + pd.DateOffset(years=4)
 
error_mask = (oc2['4bday'] > collection_end) & (oc2['SDQ_SCORE'].notna() | oc2['SDQ_REASON'].notna())
 
error_rows = oc2[error_mask]
 
error_rows

,CHILD,DOB,SDQ_SCORE,SDQ_REASON,CONVICTED,HEALTH_CHECK,IMMUNISATIONS,TEETH_CHECK,HEALTH_ASSESSMENT,SUBSTANCE_MISUSE,INTERVENTION_RECEIVED,INTERVENTION_OFFERED,4bday
2675,bad_child,2024-12-05,oops,bad,no,yes,maybe,i,don't,know,possibly,yes,2028-12-05


In [72]:
# 189 - Child is aged 17 years or over at the beginning of the year, but an Strengths
#  and Difficulties (SDQ) score or a reason for no Strengths and Difficulties (SDQ) score has been completed.
 
# 01/04/2019
# age is 17

year_start = pd.to_datetime('01/04/2019', format='%d/%m/%Y')

oc2['17bd'] = oc2['DOB'] + pd.DateOffset(years=17)

error_mask = (oc2['17bd'] < year_start) & oc2['SDQ_SCORE'].notna() | oc2['SDQ_REASON'].notna()

error_rows = oc2[error_mask]
error_rows 

,CHILD,DOB,SDQ_SCORE,SDQ_REASON,CONVICTED,HEALTH_CHECK,IMMUNISATIONS,TEETH_CHECK,HEALTH_ASSESSMENT,SUBSTANCE_MISUSE,INTERVENTION_RECEIVED,INTERVENTION_OFFERED,4bday,17bd
1,384923,2011-09-09,NaN,SDQ1,1,1,0,0,0,1,0,0,2015-09-09,2028-09-09
2,709901,2013-02-14,NaN,SDQ4,1,0,1,1,1,1,0,1,2017-02-14,2030-02-14
4,710229,2002-04-15,NaN,SDQ5,1,0,1,1,0,1,1,0,2006-04-15,2019-04-15
5,733831,2013-07-06,NaN,SDQ5,0,1,1,1,0,0,1,0,2017-07-06,2030-07-06
6,69112,2010-09-11,NaN,SDQ1,0,1,1,1,0,1,1,0,2014-09-11,2027-09-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,558581,2001-07-05,37,NaN,0,1,0,0,1,1,0,1,2005-07-05,2018-07-05
2668,937984,2008-05-08,NaN,SDQ4,0,1,1,0,1,0,0,0,2012-05-08,2025-05-08
2669,942735,2010-10-27,NaN,SDQ3,0,1,0,1,1,1,1,1,2014-10-27,2027-10-27
2673,238470,2013-11-21,NaN,SDQ2,0,0,0,1,1,1,1,0,2017-11-21,2030-11-21


In [ ]:
 # 189 - Child is aged 17 years or over at the beginning of the year, but an Strengths
#  and Difficulties (SDQ) score or a reason for no Strengths and Difficulties (SDQ) score has been completed.

collection_start = pd.to_datetime('01/04/2019', dayfirst=True)

over_17_start = oc2['DOB'] + pd.DateOffset(years=17) < collection_start
sdq_score_reason = oc2['SDQ_SCORE'].notna() | oc2['SDQ_REASON'].notna()

error_mask = over_17_start & sdq_score_reason

error_rows = oc2[error_mask]
error_rows

The is in function tests whether a value is in a lit. this can eb  list witgh a single value, but it must be a list. 